In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import datetime
import json
from pprint import pprint
from IPython.core import display as ICD  # to print multiple nice pandas tables
from collections import defaultdict
import logging
import numpy as np
import io
import gensim

In [8]:
import datasets

In [9]:
class LSABackprob(object):
    def __init__(self):
        pass
    
    def train(self, data):
        pass
    
    def predict(self, data):
        pass
    
    def score(self, data):
        pass
    
from sklearn.linear_model import LogisticRegression
class SimpleModel(object):
    def __init__(self):
        pass

    def fit(self, X, Y):
        self.dictionary = gensim.corpora.Dictionary(X)
        bow = list(map(self.dictionary.doc2bow, X))
        self.lsi = gensim.models.LsiModel(bow, id2word=self.dictionary)
        corpus = gensim.matutils.corpus2dense(self.lsi[bow], self.lsi.num_topics).T
        self.cls = LogisticRegression()
        self.cls.fit(corpus, Y)
    
    def predict(self, X):
        bow = list(map(self.dictionary.doc2bow, X))
        corpus = gensim.matutils.corpus2dense(self.lsi[bow], self.lsi.num_topics).T
        Yhat = self.cls.predict(corpus)
        return Yhat
    
    def score(self, X, Y):
        Yhat = self.predict(X)
        return 1-((Yhat-Y)**2).mean()
    
    def update(self, X, Y):
        pass

In [10]:
import datasets

In [11]:
dataset = datasets.CRDataset()
model = SimpleModel()
model.fit(dataset.samples, dataset.labels)
model.score(dataset.samples, dataset.labels)

0.7849006622516557

In [12]:
dataset = datasets.MPQADataset()
model = SimpleModel()
model.fit(dataset.samples, dataset.labels)
model.score(dataset.samples, dataset.labels)

0.7574957571186121

In [13]:
dataset = datasets.SUBJDataset()
model = SimpleModel()
model.fit(dataset.samples, dataset.labels)
model.score(dataset.samples, dataset.labels)

0.8821

In [27]:
from scipy.stats import logistic

class Classifier(object):
    def __init__(self, alpha=0.01):
        self.alpha = alpha

    def fit(self, X, Y, epoch=100):
        self.w = np.random.random(X.shape[1])*0.1 - 0.05
        self.b = np.random.random()*0.1 - 0.05
        for e in range(epoch):
            self.update(X, Y)
    
    def update(self, X, Y):
        h = (X*self.w).sum(axis=1) + self.b
        Yhat = logistic.cdf(h)
        dif = Yhat-Y
        sigma = dif * Yhat * (1-Yhat)
        dw = (sigma.reshape(-1,1)* X).sum(axis=0)
        db = sigma.sum()
        self.b -= db
        self.w -= dw
        
    def gradient(X, Y):
        pass
    
    def predict(self, X):
        h = (X*self.w).sum(axis=1) + self.b
        Yhat = logistic.cdf(h)
        return (Yhat>0.5)+0

In [33]:
n, d = 100, 5
X = np.random.random((n, d))-0.5
Y = ((X.sum(axis=1) + np.random.random(n)* - 0.5)>0)+0

cls = Classifier()
cls.fit(X, Y)

In [34]:
1-((cls.predict(X)-Y)**2).mean()

0.95